## Imports

In [ ]:
import copy
import gpxpy

import numpy as np
from sklearn.metrics import DistanceMetric

# import xml.etree.ElementTree as et
from plotly.offline import iplot, plot, init_notebook_mode
import plotly.graph_objs as go

init_notebook_mode()

## Loading

In [ ]:
with open("activity_data/250721_morning_run_original.gpx", "r") as gpx_file:
    gpx_initial = gpxpy.parse(gpx_file)

gpx = copy.deepcopy(gpx_initial)

In [ ]:
points = gpx.tracks[0].segments[0].points

In [ ]:
points_radians = np.array(
    [[point.latitude * np.pi / 180, point.longitude * np.pi / 180] for point in points]
)

## Chart data

### Compute local correction factor

In [ ]:
dist = DistanceMetric.get_metric("haversine")
local_point = points_radians[0]

epsilon = 0.00001

ddist_lat = dist.pairwise(np.array([local_point, local_point + [epsilon, 0]]))
ddist_lon = dist.pairwise(np.array([local_point, local_point + [0, epsilon]]))
correction_factor = ddist_lat[0, 1] / ddist_lon[0, 1]

print(f"map projection correction_factor = {correction_factor:.3f}")

### Chart

In [ ]:
data_labels = [
    f"id {ix}:({points[ix].latitude:.7f}, {points[ix].longitude:.7f})" for ix in range(len(points))
]
print(data_labels[:4])

trace = go.Scatter(
    x=points_radians[:, 1],
    y=points_radians[:, 0] * correction_factor,
    text=data_labels,
)

data = [trace]

layout = go.Layout(
    yaxis=dict(scaleanchor="x", scaleratio=1),
    height=600,
)

iplot(go.Figure(data=data, layout=layout))

## Repair data

### Start and end of bad segment

In [ ]:
start_ix = 1633
end_ix = 1680

### Store old data

In [ ]:
bad_points = copy.deepcopy(points[start_ix : end_ix + 1])
bad_points_labels = [
    f"id {ix + start_ix}:({bad_points[ix].latitude:.6f}, {bad_points[ix].longitude:.6f})"
    for ix in range(len(bad_points))
]
bad_points_radians = np.array(
    [[point.latitude * np.pi / 180, point.longitude * np.pi / 180] for point in bad_points]
)

### Interpolate and update `points`

In [ ]:
start_point = np.array(
    [points[start_ix].latitude, points[start_ix].longitude, points[start_ix].elevation]
)
end_point = np.array([points[end_ix].latitude, points[end_ix].longitude, points[end_ix].elevation])

start_time = points[start_ix].time
end_time = points[end_ix].time

dpos = end_point - start_point
dt = end_time - start_time

for point in points[start_ix + 1 : end_ix]:
    p = (point.time - start_time) / dt
    new_point = start_point + p * dpos
    point.latitude = round(new_point[0], 7)
    point.longitude = round(new_point[1], 7)
    point.elevation = round(new_point[2], 1)

### Compare before and after

In [ ]:
data_labels = [
    f"id {ix}:({points[ix].latitude:.7f}, {points[ix].longitude:.7f})" for ix in range(len(points))
]
points_radians = np.array(
    [[point.latitude * np.pi / 180, point.longitude * np.pi / 180] for point in points]
)

points_trace = go.Scatter(
    x=points_radians[:, 1],
    y=points_radians[:, 0] * correction_factor,
    text=data_labels,
)

bad_points_trace = go.Scatter(
    x=bad_points_radians[:, 1],
    y=bad_points_radians[:, 0] * correction_factor,
    text=bad_points_labels,
)

data = [points_trace, bad_points_trace]

layout = go.Layout(
    yaxis=dict(scaleanchor="x", scaleratio=1),
    height=600,
)

iplot(go.Figure(data=data, layout=layout))

## Export

In [ ]:
with open("activity_data/250721_morning_run_fixed.gpx", "w") as out_file:
    out_file.write(gpx.to_xml())